In [254]:
import spacy
import re
from spacy import displacy
from collections import Counter
import en_core_web_sm

In [255]:
nlp = en_core_web_sm.load()

In [294]:
#functions for number replacement
def magic(numList):
    s = ''.join(map(str, numList))
    return int(s)
def number_re(length):
    number=[]
    for i in range(length):
        number.append(random.randint(0,9))
    return magic(number)

In [296]:
import names
import random
gender_filter_dict = {
    'male ': 'human',
    'female ': 'human',
    'he': 'it',
    'she': 'it',
    'his': 'its',
    'hers': 'its'
}
email_regex = r'(?:[a-z0-9!#$%&\'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&\'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])'

def filter_text(text, filter_gender=True, filter_dates=True):
    Replace={}
    doc = nlp(text)
    print([(X.text, X.label_) for X in doc.ents])
    
    #Replace emails
    m = re.search(email_regex,text)
    print(m.group())
    Replace[m.group()]='EMAIL'
    text = re.sub(email_regex, 'EMAIL', text)
    
    if filter_gender:
        for key in gender_filter_dict:
            text = text.replace(key, gender_filter_dict[key])
            text = text.replace(key.capitalize(), gender_filter_dict[key].capitalize())
        
        
    for X in doc.ents:
        if X.label_ is 'PERSON':
            name=[]
            for i in range(len(list(nlp(X.text)))):
                name.append(names.get_first_name())
                Replace[list(nlp(X.text))[i].text]=name[i]
            replace=" ".join(name)
            text = text.replace(X.text, replace)
        
        if X.label_ is 'GPE':
            replace='LOCATION'
            text = text.replace(X.text, replace)
            for i in range(len(list(nlp(X.text)))):
                if i==0:
                    Replace[list(nlp(X.text))[i].text]=replace
                else:
                    Replace[list(nlp(X.text))[i].text]=""
        if X.label_ is 'CARDINAL':
            replace=number_re(len(X.text))
            text = text.replace(X.text, str(replace))
            Replace[list(nlp(X.text))[i].text]=str(replace)
        if filter_dates:
            if X.label_ is 'DATE':
                replace='DATE'
                text = text.replace(X.text, 'DATE')
        
        #for i in range(list(nlp(X.text))):
            #Replace[list(nlp(X.text))[0]]=replace
    
    return text,Replace
            

In [297]:
result = filter_text('Jeffrey Renik is a 23 years old police officer, he is very good at communicating. He lives in Texas America and his credit card number is 444333555. The email is rDreez@nuts.com')
result

[('Jeffrey Renik', 'PERSON'), ('23 years old', 'DATE'), ('Texas', 'GPE'), ('America', 'GPE'), ('444333555', 'CARDINAL')]
reez@nuts.com


('Harold Jerry is a DATE police officer, it is very good at communicating. It lives in LOCATION LOCATION and its credit card number is 882561424. Tit email is rDEMAIL',
 {'reez@nuts.com': 'EMAIL',
  'Jeffrey': 'Harold',
  'Renik': 'Jerry',
  'Texas': 'LOCATION',
  'America': 'LOCATION',
  '444333555': '882561424'})

In [202]:
#Generate replacement dictionary
import pandas as pd
dialogue=pd.read_json ("./Crowd-Computing-Group-5-sjoerd-gcp-speech-to-text/gcp_results.json")
text=" ".join(list(dialogue.words.values))
result,Replace = filter_text(text)
Replace

[('Pakistan', 'GPE'), ('Rebrand', 'PERSON'), ('login War Machine', 'PERSON'), ('Iron Patriot', 'PERSON')]


{'Pakistan': 'LOCATION',
 'Rebrand': 'Bradley',
 'login': 'Pablo',
 'War': 'Melody',
 'Machine': 'Doyle',
 'Iron': 'Alicia',
 'Patriot': 'Jason'}

In [257]:
#replace the identified entities
for i in range(len(dialogue)):
    if (dialogue.words[i] in Replace.keys()):
        dialogue.words[i]=Replace[dialogue.words[i]]
    if (dialogue.words[i] in gender_filter_dict.keys()):
        dialogue.words[i]=gender_filter_dict[dialogue.words[i]]

In [258]:
#transfered to json
dialogue.to_json(r'transfered.json')